In [1]:
from gymnax_exchange.jaxob.JaxOrderBookArrays import get_vision_L2_state
from gymnax_exchange.jaxob.jaxob_config import JAXLOB_Configuration
import numpy as np
import jax.numpy as jnp
from gymnax_exchange.jaxlobster.data_loading import *

In [2]:
message_file=load_message_df("AMZN_2012-06-21_34200000_57600000_message_10.csv")

In [3]:
message_file=load_message_df("AMZN_2012-06-21_34200000_57600000_message_10.csv")
orderbook_file=load_orderbook_df("AMZN_2012-06-21_34200000_57600000_orderbook_10.csv")

In [4]:
print(orderbook_file.head())
print(message_file.head())

  <ask_price_1> <ask_size_1> <bid_price_1> <bid_size_1> <ask_price_2>  \
0       2239500          100       2231800          100       2239900   
1       2239500          100       2238100           21       2239900   
2       2239500          100       2238100           21       2239600   
3       2239500          100       2238100           21       2239600   
4       2239500          100       2238100           21       2239600   

  <ask_size_2> <bid_price_2> <bid_size_2> <ask_price_3> <ask_size_3>  ...  \
0          100       2230700          200       2240000          220  ...   
1          100       2231800          100       2240000          220  ...   
2           20       2231800          100       2239900          100  ...   
3           20       2237500          100       2239900          100  ...   
4           20       2237500          100       2239900          100  ...   

  <bid_price_8> <bid_size_8> <ask_price_9> <ask_size_9> <bid_price_9>  \
0       2202500         5

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 print(orderbook_file.head())                                                                 │
│ ❱ 2 print(message_file.head())                                                                   │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'head'

In [ ]:
a=np.array([[[1,2,3],[4,5,65]],[[7,8,9],[10,11,12]]])

In [ ]:
bids=np.array([[100.0,5],[99.5,10]])
asks=np.array([[100.5, 8], [101.0,12]])

n_levels=2
# Example test case for get_vision_L2_state function
vision_L2 = get_vision_L2_state(asks, bids, n_levels,JAXLOB_Configuration)

# FIXED: Extract data using proper 3D array indexing
# vision_L2 has shape (n_levels, 2, 2)
# Axis 0: price levels, Axis 1: features (0=price, 1=volume), Axis 2: sides (0=ask, 1=bid)
extracted_asks = vision_L2[:, :, 0]  # All levels, all features, ask side
extracted_bids = vision_L2[:, :, 1]  # All levels, all features, bid side

expected_asks = np.array([[100.5, 8], [101.0, 12]])
expected_bids = np.array([[100.0, 5], [99.5, 10]])

assert np.allclose(extracted_asks, expected_asks), \
    f"Expected asks:\n{expected_asks}\nbut got:\n{extracted_asks}"
assert np.allclose(extracted_bids, expected_bids), \
    f"Expected bids:\n{expected_bids}\nbut got:\n{extracted_bids}"

print("✅ test_get_vision_L2_state PASSED!")
print(f"\nReturned shape: {vision_L2.shape}")
print(f"Extracted asks:\n{extracted_asks}")
print(f"Extracted bids:\n{extracted_bids}")

## Detailed Explanation: How the Fix Works

### The Problem
Your original code had:
```python
vision_L2['bids']  # ❌ Treats vision_L2 as a dictionary
vision_L2['asks']  # ❌ String keys don't work on arrays
```

**Why it failed:** `get_vision_L2_state()` returns a **3D JAX array**, not a dictionary with 'bids' and 'asks' keys.

---

### Understanding the Return Value

The function creates a 3D tensor using three `jnp.stack()` operations:

#### Step 1: Stack prices and volumes for each side
```python
ask_raw = jnp.stack((ask_prices, ask_vols), axis=1)   # Shape: (n_levels, 2)
bid_raw = jnp.stack((bid_prices, bid_vols), axis=1)   # Shape: (n_levels, 2)
```
Each becomes a 2D array where:
- Axis 0 = price level (0 to n_levels-1)
- Axis 1 = feature (0=price, 1=volume)

#### Step 2: Stack asks and bids together
```python
vision_l2_state = jnp.stack((ask_raw, bid_raw), axis=2)  # Shape: (n_levels, 2, 2)
```
This creates the final 3D structure where:
- **Axis 0 (dim 0):** Price level index `[0, 1, ..., n_levels-1]`
- **Axis 1 (dim 1):** Feature type `[0=price, 1=volume]`
- **Axis 2 (dim 2):** Market side `[0=ask, 1=bid]`

---

### The Solution: Proper Array Indexing

#### To extract asks (all price levels):
```python
extracted_asks = vision_L2[:, :, 0]
#                          │   │   └─ 0 = ask side
#                          │   └───── : = all features (price & volume)
#                          └───────── : = all price levels
```

#### To extract bids (all price levels):
```python
extracted_bids = vision_L2[:, :, 1]
#                          │   │   └─ 1 = bid side
#                          │   └───── : = all features (price & volume)
#                          └───────── : = all price levels
```

---

### Visualization of the Structure

For your example with `n_levels=2`:

```
vision_L2 shape: (2, 2, 2)

vision_L2 = [
    # Level 0 (best prices)
    [
        [100.5, 100.0],  # Prices: [ask_price, bid_price]
        [8.0,   5.0]     # Volumes: [ask_vol, bid_vol]
    ],
    # Level 1 (second best)
    [
        [101.0, 99.5],   # Prices: [ask_price, bid_price]
        [12.0,  10.0]    # Volumes: [ask_vol, bid_vol]
    ]
]
```

#### Accessing specific elements:
- `vision_L2[0, 0, 0]` = 100.5 (Level 0, Price, Ask)
- `vision_L2[0, 0, 1]` = 100.0 (Level 0, Price, Bid)
- `vision_L2[0, 1, 0]` = 8.0 (Level 0, Volume, Ask)
- `vision_L2[1, 0, 1]` = 99.5 (Level 1, Price, Bid)

---

### Why This Design?

This tensor structure is optimal for:
1. **Machine Learning:** Can be fed directly into CNNs or other neural networks
2. **Vectorization:** JAX can efficiently process the entire structure at once
3. **GPU Acceleration:** The regular array shape enables parallel computation
4. **Mathematical Operations:** Easy to normalize, scale, or transform all data simultaneously

The Vietnamese comment in the code confirms this: *"Tensor 3D"* - it's designed as a tensor, not a dictionary!

In [ ]:
# Practical Examples: Different Ways to Access the Data

print("=" * 60)
print("ACCESSING COMPLETE SIDES")
print("=" * 60)
print("\nAll ASK data (prices and volumes for all levels):")
print(vision_L2[:, :, 0])

print("\nAll BID data (prices and volumes for all levels):")
print(vision_L2[:, :, 1])

print("\n" + "=" * 60)
print("ACCESSING BY FEATURE TYPE")
print("=" * 60)
print("\nAll PRICES (both ask and bid sides):")
print(vision_L2[:, 0, :])

print("\nAll VOLUMES (both ask and bid sides):")
print(vision_L2[:, 1, :])

print("\n" + "=" * 60)
print("ACCESSING SPECIFIC PRICE LEVELS")
print("=" * 60)
print("\nBest prices (Level 0) - all features, both sides:")
print(vision_L2[0, :, :])
print("  → Ask: price=", vision_L2[0, 0, 0], ", volume=", vision_L2[0, 1, 0])
print("  → Bid: price=", vision_L2[0, 0, 1], ", volume=", vision_L2[0, 1, 1])

print("\nSecond best prices (Level 1) - all features, both sides:")
print(vision_L2[1, :, :])
print("  → Ask: price=", vision_L2[1, 0, 0], ", volume=", vision_L2[1, 1, 0])
print("  → Bid: price=", vision_L2[1, 0, 1], ", volume=", vision_L2[1, 1, 1])

print("\n" + "=" * 60)
print("ACCESSING INDIVIDUAL VALUES")
print("=" * 60)
print(f"\nBest ask price: {vision_L2[0, 0, 0]}")
print(f"Best bid price: {vision_L2[0, 0, 1]}")
print(f"Spread: {vision_L2[0, 0, 0] - vision_L2[0, 0, 1]}")
print(f"\nTotal ask volume: {np.sum(vision_L2[:, 1, 0])}")
print(f"Total bid volume: {np.sum(vision_L2[:, 1, 1])}")

## 📝 Summary: Key Takeaways

### What Changed in the Fix

| Original (Wrong) | Fixed (Correct) |
|-----------------|-----------------|
| `vision_L2['bids']` | `vision_L2[:, :, 1]` |
| `vision_L2['asks']` | `vision_L2[:, :, 0]` |
| Dictionary access | Array indexing |
| Would cause TypeError | Works correctly ✅ |

### The Three Critical Facts

1. **Return Type:** `get_vision_L2_state()` returns a **JAX array**, not a dictionary
2. **Shape:** The array has shape `(n_levels, 2, 2)` - a 3D tensor
3. **Index Order:** `[level, feature, side]` where:
   - `level`: 0 to n_levels-1 (price level)
   - `feature`: 0=price, 1=volume
   - `side`: 0=ask, 1=bid

### Quick Reference Cheat Sheet

```python
# Extract complete sides
all_asks = vision_L2[:, :, 0]  # Shape: (n_levels, 2)
all_bids = vision_L2[:, :, 1]  # Shape: (n_levels, 2)

# Extract by feature type
all_prices = vision_L2[:, 0, :]  # Shape: (n_levels, 2)
all_volumes = vision_L2[:, 1, :]  # Shape: (n_levels, 2)

# Get best level (top of book)
best_level = vision_L2[0, :, :]  # Shape: (2, 2)

# Get individual values
best_ask_price = vision_L2[0, 0, 0]
best_bid_price = vision_L2[0, 0, 1]
best_ask_volume = vision_L2[0, 1, 0]
best_bid_volume = vision_L2[0, 1, 1]
```

### Why This Matters for Your Trading System

This tensor structure is designed for **Vision Transformers** (ViT) in your MARL system:
- Each price level can be treated as a "pixel" or "patch"
- The 2 features (price, volume) are like color channels
- The 2 sides (ask, bid) represent different data streams
- Perfect for CNN/ViT architectures that expect regular grid inputs

In [ ]:
# Let's inspect what get_vision_L2_state actually returns
print("Type of vision_L2:", type(vision_L2))
print("Shape of vision_L2:", vision_L2.shape)
print("\nFull output:")
print(vision_L2)
print("\n--- Understanding the structure ---")
print("Dimension 0: n_levels (price levels)")
print("Dimension 1: Features (0=Price, 1=Volume)")
print("Dimension 2: Channels (0=Ask, 1=Bid)")
print("\n--- Accessing data ---")
print("All asks (vision_L2[:, :, 0]):")
print(vision_L2[:, :, 0])
print("\nAll bids (vision_L2[:, :, 1]):")
print(vision_L2[:, :, 1])

## Critical Implementation Detail: Return Type

**The issue:** `get_vision_L2_state` returns a **3D JAX array**, NOT a dictionary!

### Return Structure: `shape = (n_levels, 2, 2)`

The function stacks data in this specific way:
```python
ask_raw = jnp.stack((ask_prices, ask_vols), axis=1)   # (n_levels, 2)
bid_raw = jnp.stack((bid_prices, bid_vols), axis=1)   # (n_levels, 2)
vision_l2_state = jnp.stack((ask_raw, bid_raw), axis=2)  # (n_levels, 2, 2)
```

### Indexing Scheme:
- **Axis 0 (Dimension 0):** Price level index (0 to n_levels-1)
- **Axis 1 (Dimension 1):** Feature type
  - `0` = Price
  - `1` = Volume
- **Axis 2 (Dimension 2):** Side
  - `0` = Ask side
  - `1` = Bid side

### How to access the data:
- `vision_L2[:, :, 0]` → All ask data (prices & volumes)
- `vision_L2[:, :, 1]` → All bid data (prices & volumes)  
- `vision_L2[:, 0, :]` → All prices (both sides)
- `vision_L2[:, 1, :]` → All volumes (both sides)
- `vision_L2[0, :, :]` → First price level (both features, both sides)

In [ ]:
# CORRECTED TEST - Understanding the 3D structure

# Expected structure:
# vision_L2[level_idx, feature_idx, side_idx]
# where:
#   level_idx: 0 to n_levels-1
#   feature_idx: 0=price, 1=volume
#   side_idx: 0=ask, 1=bid

# Extract asks and bids properly
extracted_asks = vision_L2[:, :, 0]  # All levels, all features, ask side
extracted_bids = vision_L2[:, :, 1]  # All levels, all features, bid side

# Define expected values
expected_asks_array = np.array([[100.5, 8], [101.0, 12]])
expected_bids_array = np.array([[100.0, 5], [99.5, 10]])

# Test with proper array comparison
assert np.allclose(extracted_asks, expected_asks_array), \
    f"Expected asks:\n{expected_asks_array}\nbut got:\n{extracted_asks}"
assert np.allclose(extracted_bids, expected_bids_array), \
    f"Expected bids:\n{expected_bids_array}\nbut got:\n{extracted_bids}"

print("✅ test_get_vision_L2_state PASSED!")
print("\nExtracted asks:")
print(extracted_asks)
print("\nExtracted bids:")
print(extracted_bids)

## Why the Original Test Failed

### ❌ Original (Wrong) Approach:
```python
vision_L2['bids']  # Trying to access like a dictionary
```

**Problem:** `vision_L2` is a JAX array, not a dict. JAX arrays use numeric indexing `[i, j, k]`, not string keys.

### ✅ Correct Approach:
```python
vision_L2[:, :, 1]  # Access bid side using numeric index
```

### Key Takeaway:
The function implementation uses `jnp.stack()` three times:
1. Stack prices & volumes → `(n_levels, 2)` 
2. Do this for both asks and bids separately
3. Stack asks and bids along axis=2 → `(n_levels, 2, 2)`

This creates a **tensor/array structure**, not a dictionary structure. The function comment even says "Tensor 3D" in Vietnamese!

# 📂 LOBSTER File Loading System

## Overview
The codebase uses a multi-layer system to load LOBSTER (Limit Order Book System - The Electronic Record) data files into the trading environment.

---

## Main Loading Classes

### 1. **`LoadLOBSTER`** - Basic Loader
**Location:** [gymnax_exchange/jaxlobster/lobster_loader.py](gymnax_exchange/jaxlobster/lobster_loader.py#L50)

**Purpose:** Loads LOBSTER CSV files and slices them into fixed-size episode windows

**Key Features:**
- Loads message files and orderbook files separately
- Splits data into non-overlapping windows
- Pads windows to same size for batch processing

### 2. **`LoadLOBSTER_resample`** - Advanced Loader ⭐
**Location:** [gymnax_exchange/jaxlobster/lobster_loader.py](gymnax_exchange/jaxlobster/lobster_loader.py#L287)

**Purpose:** More sophisticated loader with resampling and parallel loading capabilities

**This is the one actually used in your environment!**

---

## File Loading Pipeline

### Step 1: File Discovery
```python
# In LoadLOBSTER_resample.__init__()
self.datapath = datapath + f"/{stock}/{time_period}/"
self.message_files = sorted(glob(self.datapath + '*message*.csv'))
self.book_files = sorted(glob(self.datapath + '*orderbook*.csv'))
```

**File naming convention:**
- Message files: `{STOCK}_{DATE}_{START}_{END}_message_{LEVELS}.csv`
- Orderbook files: `{STOCK}_{DATE}_{START}_{END}_orderbook_{LEVELS}.csv`

Example in your repo: `AMZN_2012-06-21_34200000_57600000_message_10.csv`

### Step 2: CSV Reading
**Location:** [lobster_loader.py#L466-L550](gymnax_exchange/jaxlobster/lobster_loader.py#L466)

```python
def _load_files(self):
    # Uses pandas with parallel processing
    df_message = pd.read_csv(
        message_file, 
        usecols=range(6),  # Only first 6 columns
        dtype={0: float, 1: int, 2: int, 3: int, 4: int, 5: int},
        header=None,
        engine='c',        # Fast C parser
        low_memory=True,
        na_filter=False
    )
    
    df_book = pd.read_csv(
        book_file, 
        header=None,
        engine='c'
    )
```

**Message file columns (6 total):**
1. `time` - Timestamp (seconds.nanoseconds)
2. `type` - Message type (1=new, 2=cancel, 3=delete, 4=execution)
3. `order_id` - Order identifier
4. `qty` - Order size
5. `price` - Order price
6. `direction` - Buy(1) or Sell(-1)

### Step 3: Pre-processing
**Location:** [lobster_loader.py#L166-L195](gymnax_exchange/jaxlobster/lobster_loader.py#L166)

```python
def _pre_process_msg_ob(self, message_day, orderbook_day):
    # Split timestamp into seconds and nanoseconds
    message_day[6] = message_day[0].apply(lambda x: int(x))
    message_day[7] = ((message_day[0] - message_day[6]) * 1e9).astype(int)
    
    # Filter out unwanted message types (5,6,7)
    message_day = message_day[message_day.type.isin([1,2,3,4])]
    
    # Transform executions → limit orders on opposite side
    message_day.loc[message_day['type'] == 4, 'direction'] *= -1
    message_day.loc[message_day['type'] == 4, 'type'] = 1
    
    # Transform delete → cancel
    message_day.loc[message_day['type'] == 3, 'type'] = 2
    
    # Add trader_id field
    message_day['trader_id'] = message_day['order_id']
```

### Step 4: Window Slicing
Messages are split into episodes based on:
- **Fixed time:** e.g., 30-minute windows
- **Fixed steps:** e.g., 300 messages per window

### Step 5: Return Format
```python
return (
    loaded_msg_windows,    # Shape: (n_windows, max_steps, max_msgs_per_step, 8)
    loaded_book_windows,   # Initial orderbook states
    max_steps_arr,         # Actual steps per window
    n_windows              # Number of windows
)
```

---

## How It's Used in Your Environment

**Location:** [gymnax_exchange/jaxen/base_env.py#L154-L167](gymnax_exchange/jaxen/base_env.py#L154)

```python
# Initialize the loader
loader = LoadLOBSTER_resample(
    self.cfg.dataPath,           # e.g., "./data"
    self.cfg.alphatradePath,     # Path to AlphaTrade repo
    self.book_depth,             # Number of price levels (10)
    self.ep_type,                # "fixed_time" or "fixed_steps"
    window_length=self.episode_time,    # 1800 seconds or N steps
    n_data_msg_per_step=self.n_data_msg_per_step,  # 100
    window_resolution=self.start_resolution,
    day_start=self.day_start,    # 34200 (9:30 AM)
    day_end=self.day_end,        # 57600 (4:00 PM)
    stock=self.cfg.stock,        # "AMZN"
    time_period=self.cfg.timePeriod  # "2017Jan_oneday"
)

# Load all data
msgs, starts, ends, books, max_messages_arr = loader.run_loading()
```

---

## Alternative Loading Functions

### For Tokenization/ML Training
**Location:** [gymnax_exchange/jaxlobster/data_loading.py](gymnax_exchange/jaxlobster/data_loading.py)

```python
def load_message_df(file: str, cast_dtypes: bool = False) -> pd.DataFrame
def load_orderbook_df(file: str, cast_dtypes: bool = False) -> pd.DataFrame
```

These are simpler loaders for direct CSV access without windowing.

In [ ]:
# Example: How to load LOBSTER files directly

from gymnax_exchange.jaxlobster.lobster_loader import LoadLOBSTER_resample

# Your LOBSTER files in the root directory match this pattern:
# AMZN_2012-06-21_34200000_57600000_message_10.csv
# AMZN_2012-06-21_34200000_57600000_orderbook_10.csv

# To load these files:
data_path = "e:/OneDrive - 7t12kn/Documents/GitHub/ViT-MARL"  # Root directory
alphatrade_path = "e:/OneDrive - 7t12kn/Documents/GitHub/ViT-MARL"  # Same for your setup

loader = LoadLOBSTER_resample(
    datapath=data_path,
    atpath=alphatrade_path,
    n_Levels=10,                    # Your files have 10 price levels
    type_="fixed_time",             # Window by time
    window_length=1800,             # 30 minutes per episode
    window_resolution=60,           # Windows can start every 60 seconds
    n_data_msg_per_step=100,       # Process 100 messages per step
    day_start=34200,                # 9:30 AM in seconds (9.5 * 3600)
    day_end=57600,                  # 4:00 PM in seconds (16 * 3600)
    stock="AMZN",                   # Stock ticker
    time_period="2012-06-21"        # Will look for files with this pattern
)

print("\n🔍 Searching for files in:", loader.datapath)
print(f"📨 Found {len(loader.message_files)} message files")
print(f"📚 Found {len(loader.book_files)} orderbook files")

if loader.message_files:
    print("\n📄 Message files:")
    for f in loader.message_files:
        print(f"  - {f}")
        
if loader.book_files:
    print("\n📄 Orderbook files:")
    for f in loader.book_files:
        print(f"  - {f}")

In [ ]:
# Since your files are in the root directory, let's load them directly with pandas

import pandas as pd
import os

# Check what files exist in the root
root_dir = "e:/OneDrive - 7t12kn/Documents/GitHub/ViT-MARL"
csv_files = [f for f in os.listdir(root_dir) if f.endswith('.csv')]

print("📁 CSV files in root directory:")
for f in csv_files:
    print(f"  - {f}")

# Load the message file
message_file = os.path.join(root_dir, "AMZN_2012-06-21_34200000_57600000_message_10.csv")
orderbook_file = os.path.join(root_dir, "AMZN_2012-06-21_34200000_57600000_orderbook_10.csv")

print(f"\n📨 Loading message file...")
dtypes = {0: float, 1: int, 2: int, 3: int, 4: int, 5: int}
df_message = pd.read_csv(message_file, usecols=range(6), dtype=dtypes, header=None)

print(f"✅ Loaded {len(df_message)} messages")
print(f"\nMessage file columns: {list(df_message.columns)}")
print(f"Shape: {df_message.shape}")
print(f"\nFirst 5 messages:")
df_message.columns = ['time', 'type', 'order_id', 'qty', 'price', 'direction']
print(df_message.head())

print(f"\n📚 Loading orderbook file...")
df_orderbook = pd.read_csv(orderbook_file, header=None)

print(f"✅ Loaded {len(df_orderbook)} orderbook states")
print(f"Shape: {df_orderbook.shape}")
print(f"Columns (10 levels × 4 fields): {df_orderbook.shape[1]} columns")
print(f"\nFirst orderbook state (4 values per level: ask_price, ask_vol, bid_price, bid_vol):")
print(df_orderbook.iloc[0])

## 📊 Understanding Your LOBSTER Data

### File Format Summary

**Your files contain:**
- **269,748 messages** over the trading day
- **10 price levels** of orderbook depth
- Time period: **34200s to 57600s** (9:30 AM to 4:00 PM)
- Stock: **AMZN** on **2012-06-21**

---

### Message File Structure (6 columns)

| Column | Name | Type | Description |
|--------|------|------|-------------|
| 0 | `time` | float | Timestamp (seconds.nanoseconds) |
| 1 | `type` | int | Message type (1=new, 2=cancel, 3=delete, 4=execution, 5=hidden) |
| 2 | `order_id` | int | Unique order identifier |
| 3 | `qty` | int | Order quantity (shares) |
| 4 | `price` | int | Price in hundredths of cents (2238200 = $223.82) |
| 5 | `direction` | int | Buy(1) or Sell(-1) |

### Orderbook File Structure (40 columns)

The orderbook file has **4 columns per price level × 10 levels = 40 columns:**

For each level `i` (0-9):
- Column `4i`: Ask price
- Column `4i+1`: Ask volume
- Column `4i+2`: Bid price  
- Column `4i+3`: Bid volume

Example for Level 0 (best prices):
```
Columns 0-3: [2239500, 100, 2231800, 100]
             [ask_price, ask_vol, bid_price, bid_vol]
             [$223.95 × 100, $223.18 × 100]
```

---

### Key Loading Functions in Your Codebase

| Function/Class | Location | Purpose |
|----------------|----------|---------|
| `LoadLOBSTER_resample` | [lobster_loader.py#L287](gymnax_exchange/jaxlobster/lobster_loader.py#L287) | Main loader with windowing (used in env) |
| `LoadLOBSTER` | [lobster_loader.py#L50](gymnax_exchange/jaxlobster/lobster_loader.py#L50) | Basic loader |
| `load_message_df()` | [data_loading.py#L31](gymnax_exchange/jaxlobster/data_loading.py#L31) | Direct message CSV loading |
| `load_orderbook_df()` | [data_loading.py#L49](gymnax_exchange/jaxlobster/data_loading.py#L49) | Direct orderbook CSV loading |
| `_load_files()` | [lobster_loader.py#L466](gymnax_exchange/jaxlobster/lobster_loader.py#L466) | Parallel CSV loading with pandas |

---

### How Data Flows into Your Environment

```
LOBSTER CSV Files
        ↓
LoadLOBSTER_resample (with parallel loading)
        ↓
_pre_process_msg_ob() (clean & transform)
        ↓
_slice_day_no_overlap() (split into windows)
        ↓
BaseEnv initialization
        ↓
JAX arrays ready for RL training
```